This is based on this code: https://github.com/codekansas/keras-language-modeling/blob/master/keras_models.py

In [1]:
#%load_ext autoreload
#%autoreload 2

In [3]:
import numpy as np
import Settings
import logging
import datetime

from collections import defaultdict
from load_data import load_process_essays

## Load and Pre-Process Essays

In [6]:
import pickle
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [31]:
training_pickled

'/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/training.pl'

In [7]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [8]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-05-21 14:55:27.284925


In [9]:
from numpy.random import shuffle
shuffle(tagged_essays)

## Get Max Sequence Length, Generate All Ids

In [11]:
generator = idGen(seed=1) # important as we zero pad sequences

maxlen = 0
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
        maxlen = max(maxlen, len(sentence) + 2)

def ids2tags(ids):
    return [generator.get_key(j) for j in ids]  

def lbls2tags(ixs):
    return [ix2tag[ix] for ix in ixs]
        
maxlen

93

## Create Train - Test Split

In [17]:
def get_unique_words(tessays):
    wds = set()
    for essay in tessays:
        for sentence in essay.sentences:
            for word, tags in sentence:
                wds.add(word)
    return wds

In [37]:
%%time

folds = cross_validation(tagged_essays, CV_FOLDS)
diff_counts = []
vd_lens = []
for i, (essays_TD, essays_VD) in enumerate(folds):
    td_wds = get_unique_words(essays_TD)
    vd_wds = get_unique_words(essays_VD)
    diff = vd_wds - td_wds
    diff_counts.append(len(diff))
    vd_lens.append(len(vd_wds))
    print("|TD|={td_len} |VD|={vd_len} |diff|={len_diff}".format(td_len=len(td_wds), vd_len=len(vd_wds), len_diff=len(diff)))
    
print("")
print(float(sum(diff_counts)) / sum(vd_lens))
print("")

|TD|=1618 |VD|=1210 |diff|=23
|TD|=1616 |VD|=1208 |diff|=25
|TD|=1615 |VD|=1197 |diff|=26
|TD|=1618 |VD|=1199 |diff|=23
|TD|=1612 |VD|=1194 |diff|=29

0.020972037283621837

CPU times: user 148 ms, sys: 3.15 ms, total: 151 ms
Wall time: 150 ms


## Load The Test Dataset

In [33]:
test_folder = root_folder + "Test" + "/"
test_config = get_config(test_folder)
test_essays = load_process_essays(**test_config)

226 files found
226 essays processed


In [38]:
td_wds = get_unique_words(tagged_essays)
vd_wds = get_unique_words(test_essays)
diff = vd_wds - td_wds
print("|TD|={td_len} |VD|={vd_len} |diff|={len_diff}".format(td_len=len(td_wds), vd_len=len(vd_wds), len_diff=len(diff)))
print(float(len(diff))/len(vd_wds))

|TD|=1641 |VD|=927 |diff|=36
0.038834951456310676
